# Dealing with real data



In [ ]:
import json
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()

## Loading the data

Let us assume you start with the excel file downloaded. You will first have to convert it to a more standard `.csv` format before being able to read it with pandas (actually it is possible to [read excel file](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html) directly, but this is less standard and require additional installation).

- Open the Excel file in Excel and `Save as` with the format being selected as `csv utf-8`. (if you do not have excel there is a csv already available next to the excel file)
- Try loading it with the following command :

In [ ]:
df = pd.read_csv('my_csv_export.csv')

You will notice that there is a `ParserError` reading the file (assuming you have saved it properly and are referring to its name properly). This is because the file is actually not directly properly formatted.

Open the `csv` file (right-click, open with) directly in a text editor (not Word or Wordpad!, but a pure text editor such as the Notepad, or much better VisualStudioCode).

Remember that a `csv` should have the header line of the column names as the first row, and its values delimited by `,`. Looking at the file, and exploring the options of the `read_csv` function, can you correct the line below to properly load the data?

In [ ]:
df = pd.read_csv('my_csv_export.csv',
            #header=??,
            #delimiter=??
           )

In [ ]:
df

## Cleaning the data

By default, the read data is not exactly clean so let us improve that.

### Removing empty columns

First, you might have noticed that there is a very large number of empty columns, which is an artifact of the export and the fact that there are very long values in the original excel file, which makes excel creates arbitrary empty columns to fit the long values in the display.

This can be seen by listing the (more than 250) columns.

In [ ]:
# Gets the index object representing the names of all the columns
df.columns

In order to remove them, we first need to detect them. Detecting missing values can be done by directly using the method `isna()`, but it will return another `pd.Series` describing missing value at every position, and not a single boolean for the whole column. In order to find columns where there is no interesting value in, have a look at the methods [`any()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.any.html) or [`all()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.all.html)

In [ ]:
def is_column_empty(serie):
    # Your code here
    return False

print("should be False : ", is_column_empty(df['School']))
print("should be True : ", is_column_empty(df['Unnamed: 248']))

Using the previous function, we can now remove the unnecessary columns. For this, simply iterate over the column names (with `df.columns` as seen above), and for each of them: test if they are empty and delete them if necessary.

**HINT:** deleting column is the same syntax as deleting elements of a dictionnary.

In [ ]:
# Your code here


In [ ]:
# This should evaluate to True if only the non-empty columns are left
set(df.columns) == set(['School', 'Author', 'Actual Attributon', 'Place', 'Room', 'Title',
       'Date', 'Date Acquired', 'Date Sold-Tranferred', 'Material / Technique',
       'Size (cm)', 'N. Inv. Le Brun', 'Actual Location', 'ICONCLASS', '#',
       '#.1', '#.2', '#.3', 'Link', 'Unnamed: 23'])

In [ ]:
df

### Standardizing types

Pandas tries to convert the columns to the best data types and tries to separate categorical and numerical columns.

Looking at the data in the `DataFrame` and the current data type (printed with the command below). Identify which column is not properly identified as being numerical.

In [ ]:
# Gives the listing of current columns with their associated data type
df.dtypes

Indeed, you can see directly in Excel or here that there is one value in that column can not be converted to numeric directly (line 14 to be exact)

In [ ]:
df['Date Acquired'][10:20]

This can create issues, for instance it is impossible to plot the joint plot of the dates of acquisition and selling. The following plotting command will give the appropriate error.

In [ ]:
sns.jointplot('Date Acquired', 'Date Sold-Tranferred', data=df)

In order to convert a column to a numeric one, the function `pd.to_numeric` can be used. By default, it will fail because of the string value, look at the options of that function so that the wrong value is automatically dropped (by being converted to `np.nan`). 

Override the column in the `DataFrame` by assigning the converted column with the same name. (Reminder: it is still the same syntax as a dictionnary assignment)

In [ ]:
# Your code here

In [ ]:
# Should be True now
df['Date Acquired'].dtype == np.float

Now the jointplot should display properly.

This visualization does not actually bring a lot of information as it only display the data points where both the acquisition date AND the selling date is known, which does not represent many data points.

However, looking more closely at the previous plot, you might notice that some configurations do not seem possible and show wrong data. Can an object be sold before it is acquired???

In [ ]:
sns.jointplot('Date Acquired', 'Date Sold-Tranferred', data=df)

Using filtering, display the two cases where the data is wrong (so that one could correct it in the original data file for instance, not necessary to correct it for the sake of this exercise though).

In [ ]:
# Your code here

## Extracting the main Iconography

Each record has a full ICONCLASS specification. [ICONCLASS](https://en.wikipedia.org/wiki/Iconclass) is a hierarchical iconographic classification system that is very involved. Here we take a simplistic approach and just extract the top-level (first digit) of the specification.

### Installation of the iconclass python package

In order to work with a more readable version of the ICONCLASS system, someone nicely made a python package that maps the obscure definitions to their textual representation.

Installation can directly be done with the following command (special jupyter notebook shell command, starting with `!`) calling the python package manager `pip`

In [ ]:
!pip install iconclass

In [ ]:
# Import the newly installed package (needs to download a small database at the first import)
import iconclass
# Example usage
iconclass.get('1')

This package allow us to easily convert a ICONCLASS id to a dictionnary containing all the necessary information. The sub-classes in the tree of classification, the textual and keyword representation in multiple languages, and the parent classes.

In [ ]:
iconclass.get('11A')

In [ ]:
iconclass.get('11A(+5)')

### Simple parsing: the ICONCLASS column

Using this package we can now parse, for each record with an ICONCLASS value, its main category.

In [ ]:
def extract_main_iconography(iconclass_string):
    # Your code here
    return 'Nothing'

print('Should be "Religion and Magic" : ', extract_main_iconography("11 M 33"))
print('Should be "Classical Mythology and Ancient History" : ', extract_main_iconography("94 M 22 1"))

Now use the `apply` method on the `ICONCLASS` column to create a new column with the textual value of the main iconography. Careful that you might need to filter the `NaN` values of the column before using `apply`.

Add the created column to the original `DataFrame` with a name of your choice.

In [ ]:
# Your code here

Using seaborn `countplot` (or any other method), plot the distribution of records per iconography, what seemed to be the two most important genres?

(Hint: depending on how you do it the labels of the plot might overlap each other making them unreadable. To rotate them, use `plt.xticks(rotation=90)`)

In [ ]:
# Your code here

## Harder parsing : extracting the size information

Since not every artwork is the same size, we want to not only measure the number of artworks owned at that time but also the surface they occupied.

For this let us create a function that takes the information of size in the data and tries to separate the height and the width information. Using `split` and `replace` you should be able to parse all but 4 valid values.

In [ ]:
def parse_size(s):
    try:
        # Your code here
        h, w = 0, 0
        return h, w
    except Exception as e:
        print(f"Can not parse: {s}")
        return np.nan, np.nan
    
df['Size (cm)'].apply(parse_size)

In [ ]:
# This line is a bit complex so I give it to you directly
df['Height'], df['Width'] = zip(*df['Size (cm)'].apply(parse_size))

# Checking that the column has proper typing
print('Should be True : ', df['Height'].dtype == np.float)
print('Should be True : ', df['Width'].dtype == np.float)

Here is another variant in order to do the same thing but sligthly differently. Instead of using `apply` only on a column we use it direclty on the whold dataframe, and work on a full record (row). The same parsing code should then be used and we know return a dictionnary. This column of dictionnary is then `expanded` to directly make a new dataframe with the values of the two newly created columns.

In [ ]:
def parse_size_2(record):
    s = record['Size (cm)']
    try:
        # Same code as above
        h, w = 0, 0
        return {'Height': h, 'Width': w}
    except Exception as e:
        print(f"Can not parse: {s}")
        return {'Height': np.nan, 'Width': np.nan}

# Generates directly a new DataFrame with two columns 'Height' and 'Width'. It can be added with `df.merge(...)`
df.apply(parse_size_2, result_type='expand', axis='columns')

We have now separated properly the height and width of each painting. The area can then easily be computed and added as an additional column named 'Area'. Looking at the example below of how mathematical operators easily propagate on full columns, add the corresponding 'Area' column to the `DataFrame`

In [ ]:
# Example
df['Date Sold-Tranferred'] - df['Date Acquired']

In [ ]:
# Your code here

Now using the `swarmplot` of seaborn, look at the distribution of the size of paintings according to their main iconography. What can you say?

In [ ]:
# Your code here

### Custom Aggregation

In order to compute exactly the number of painting in each category and the combined areas of all the paintings in each category we need to aggregate the information.

Try to use `groupby` on the Dataframe and then apply `sum` or `count` to create aggregated series. You might have to google a bit around to understand how `groupby` works.

In [ ]:
# your code here

You can then directly plot the aggregated values with `plot(kind='bar')` on the aggregated column. Display the aggregated count of records per iconographic category, and the aggregated area per iconographic category. Do they tell different stories?

In [ ]:
# Your code here


## Last questions

### More precise analysis

Of course, there is much more to be done that we are doing here. For instance can you adapt and rerun your code so that we display the statistics based on the second level iconography and not the top-level?

What about the represented authors?

### Sampling bias

There are multiple records which are not in the final analysis, because one of the necessary field was not available or not parsed properly. Can you count the percentage of records missing in this analysis? Does it change the conclusions?

In [ ]:
# Your code here